## SpanMarker for Named Entity Recognition
[SpanMarker](github.com/tomaarsen/SpanMarkerNER) is an accessible yet powerful Python module for training Named Entity Recognition models.

In this notebook, we'll have a look at how to train an NER model using SpanMarker.

### Setup
First of all, the `span_marker` Python module needs to be installed. If we want to use [Weights and Biases](https://wandb.ai/) for logging, we can install `span_marker` using the `[wandb]` extra.

In [ ]:
%pip install span_marker
# %pip install span_marker[wandb]

### Loading the dataset
For this example, we'll load the challenging [FewNERD supervised dataset](https://huggingface.co/datasets/DFKI-SLT/few-nerd) from the Hugging Face hub using 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 131767
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 18824
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 37648
    })
})

Let's inspect some samples to get a feel for the data.

In [3]:
for sample in dataset["train"].select(range(3)):
    print(sample)

{'id': '0', 'tokens': ['Paul', 'International', 'airport', '.'], 'ner_tags': [0, 0, 0, 0], 'fine_ner_tags': [0, 0, 0, 0]}
{'id': '1', 'tokens': ['It', 'starred', 'Hicks', "'s", 'wife', ',', 'Ellaline', 'Terriss', 'and', 'Edmund', 'Payne', '.'], 'ner_tags': [0, 0, 7, 0, 0, 0, 7, 7, 0, 7, 7, 0], 'fine_ner_tags': [0, 0, 51, 0, 0, 0, 50, 50, 0, 50, 50, 0]}
{'id': '2', 'tokens': ['``', 'Time', '``', 'magazine', 'said', 'the', 'film', 'was', '``', 'a', 'multimillion', 'dollar', 'improvisation', 'that', 'does', 'everything', 'but', 'what', 'the', 'title', 'promises', "''", 'and', 'suggested', 'that', '``', 'writer', 'George', 'Axelrod', '(', '``', 'The', 'Seven', 'Year', 'Itch', '``', ')', 'and', 'director', 'Richard', 'Quine', 'should', 'have', 'taken', 'a', 'hint', 'from', 'Holden', '[', "'s", 'character', 'Richard', 'Benson', ']', ',', 'who', 'writes', 'his', 'movie', ',', 'takes', 'a', 'long', 'sober', 'look', 'at', 'what', 'he', 'has', 'wrought', ',', 'and', 'burns', 'it', '.', "''"], 'n

As you can see, this dataset contains `tokens`, `ner_tags` and a `fine_ner_tags` columns. Let's have a look at which labels these last two represent using the [Dataset features](https://huggingface.co/docs/datasets/about_dataset_features).

In [4]:
labels = dataset["train"].features["ner_tags"].feature.names
print(labels)

['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']


In [5]:
fine_labels = dataset["train"].features["fine_ner_tags"].feature.names
print(fine_labels)

['O', 'art-broadcastprogram', 'art-film', 'art-music', 'art-other', 'art-painting', 'art-writtenart', 'building-airport', 'building-hospital', 'building-hotel', 'building-library', 'building-other', 'building-restaurant', 'building-sportsfacility', 'building-theater', 'event-attack/battle/war/militaryconflict', 'event-disaster', 'event-election', 'event-other', 'event-protest', 'event-sportsevent', 'location-GPE', 'location-bodiesofwater', 'location-island', 'location-mountain', 'location-other', 'location-park', 'location-road/railway/highway/transit', 'organization-company', 'organization-education', 'organization-government/governmentagency', 'organization-media/newspaper', 'organization-other', 'organization-politicalparty', 'organization-religion', 'organization-showorganization', 'organization-sportsleague', 'organization-sportsteam', 'other-astronomything', 'other-award', 'other-biologything', 'other-chemicalthing', 'other-currency', 'other-disease', 'other-educationaldegree', '

For the purposes of this tutorial, let's stick with the `ner_tags` coarse-grained labels, but I challenge you to modify this Notebook to train for the fine labels. For the SpanMarker model, any dataset can be used as long as it has a `tokens` and a `ner_tags` column. The `ner_tags` can be annotated using the IOB, IOB2, BIOES or BILOU labeling scheme, but also regular unschemed labels like in this FewNERD example can be used.

### Initializing a `SpanMarkerModel`
A SpanMarker model is initialized via `SpanMarkerModel.from_pretrained`. This method will be familiar to those who know 🤗 Transformers. It accepts either a path to a local model or the name of a model on the [Hugging Face Hub](https://huggingface.co/models).

Importantly, the model can *either* be an encoder or an already trained and saved SpanMarker model. As we haven't trained anything yet, we will use an encoder.

Reasonable options for encoders include BERT, RoBERTa, etc., which means that the following are all good options: `"bert-base-cased"`, `"bert-large-cased"`, `"roberta-base"`, `"roberta-large"`. Not all encoders work though, they **must** allow for `position_ids` as an input argument, which disqualifies DistilBERT, T5, DistilRoBERTa, ALBERT & BART. Furthermore, using uncased models is generally not recommended, as the capitalisation can be very useful to find named entities.

We'll use `"bert-base-cased"` for this notebook. Note that we ensure that the model is placed on the GPU with `.cuda()`. If you're running this on Google Colab, be sure to set hardware accelerator to "GPU" in `Runtime` > `Change runtime type`.

In [6]:
from span_marker import SpanMarkerModel

model_name = "bert-base-cased"
model = SpanMarkerModel.from_pretrained(model_name, labels=labels, model_max_length=256).cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


For us, these warnings are expected, as we are initializing `BertModel` for a new task.

Note that we provided `SpanMarkerModel.from_pretrained` with a list of our labels. This is required when training a new model using an encoder. Furthermore, we can specify some useful configuration parameters from `SpanMarkerConfig`, such as:

* `model_max_length`: The maximum number of tokens that the model will process. If you only use short sentences for your model, reducing this number may help training and inference speeds with no loss in performance. Defaults to the encoder maximum, or 512 if the encoder doesn't have a maximum.
* `entity_max_length`: The total number of words that one entity can be. Defaults to 16.

### Training
At this point, our model is already ready for training! We can import [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) directly from 🤗 Transformers as well as the SpanMarker `Trainer`. The `Trainer` is a subclass of the 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) that simplifies some tasks for you, but otherwise it works just like the regular `Trainer`.

This next snippet shows some reasonable defaults. Feel free to adjust the batch size to a lower value if you experience out of memory exceptions.

In [7]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="my_span_marker_model",
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    push_to_hub=False,
    logging_steps=50,
    fp16=True,
    warmup_ratio=0.1,
)

Now we can create a SpanMarker `Trainer` in the same way that you would initialize a 🤗 Transformers `Trainer`. We'll train on a subsection of the data to save us some time. Amazingly, this `Trainer` will automatically create logs using exactly the logging tools that you have installed. With other words, if you prefer logging with [Tensorboard](https://www.tensorflow.org/tensorboard), all that you have to do is install it.

In [ ]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

Let's start training!

In [9]:
trainer.train()

  0%|          | 0/1598 [00:00<?, ?it/s]

{'loss': 0.9641, 'learning_rate': 1.5625e-05, 'epoch': 0.03}
{'loss': 0.0589, 'learning_rate': 3.125e-05, 'epoch': 0.06}
{'loss': 0.0402, 'learning_rate': 4.6875e-05, 'epoch': 0.09}
{'loss': 0.0316, 'learning_rate': 4.860917941585536e-05, 'epoch': 0.13}


  0%|          | 0/798 [00:00<?, ?it/s]

{'eval_loss': 0.024076074361801147, 'eval_overall_precision': 0.8170849420849421, 'eval_overall_recall': 0.32545174932718185, 'eval_overall_f1': 0.4654935386307396, 'eval_overall_accuracy': 0.847783060254239, 'eval_runtime': 56.1727, 'eval_samples_per_second': 56.825, 'eval_steps_per_second': 14.206, 'epoch': 0.13}
{'loss': 0.0255, 'learning_rate': 4.687065368567455e-05, 'epoch': 0.16}
{'loss': 0.0222, 'learning_rate': 4.513212795549374e-05, 'epoch': 0.19}
{'loss': 0.0187, 'learning_rate': 4.3393602225312935e-05, 'epoch': 0.22}
{'loss': 0.0206, 'learning_rate': 4.165507649513213e-05, 'epoch': 0.25}


  0%|          | 0/798 [00:00<?, ?it/s]

...

  0%|          | 0/798 [00:00<?, ?it/s]

{'eval_loss': 0.012467470020055771, 'eval_overall_precision': 0.7672907909820404, 'eval_overall_recall': 0.7720107650903498, 'eval_overall_f1': 0.7696435415868148, 'eval_overall_accuracy': 0.9368687383949886, 'eval_runtime': 55.1376, 'eval_samples_per_second': 57.892, 'eval_steps_per_second': 14.473, 'epoch': 0.75}
{'loss': 0.0134, 'learning_rate': 1.2100139082058415e-05, 'epoch': 0.78}
{'loss': 0.0123, 'learning_rate': 1.0361613351877608e-05, 'epoch': 0.81}
{'loss': 0.0113, 'learning_rate': 8.623087621696801e-06, 'epoch': 0.84}
{'loss': 0.0136, 'learning_rate': 6.884561891515995e-06, 'epoch': 0.88}


  0%|          | 0/798 [00:00<?, ?it/s]

{'eval_loss': 0.011955244466662407, 'eval_overall_precision': 0.7783554785287887, 'eval_overall_recall': 0.7770088427527874, 'eval_overall_f1': 0.7776815776815776, 'eval_overall_accuracy': 0.939052010855149, 'eval_runtime': 54.9137, 'eval_samples_per_second': 58.128, 'eval_steps_per_second': 14.532, 'epoch': 0.88}
{'loss': 0.0112, 'learning_rate': 5.1460361613351884e-06, 'epoch': 0.91}
{'loss': 0.0118, 'learning_rate': 3.4075104311543813e-06, 'epoch': 0.94}
{'loss': 0.0128, 'learning_rate': 1.6689847009735746e-06, 'epoch': 0.97}
{'train_runtime': 809.692, 'train_samples_per_second': 15.792, 'train_steps_per_second': 1.974, 'train_loss': 0.04753492740874595, 'epoch': 1.0}


TrainOutput(global_step=1598, training_loss=0.04753492740874595, metrics={'train_runtime': 809.692, 'train_samples_per_second': 15.792, 'train_steps_per_second': 1.974, 'train_loss': 0.04753492740874595, 'epoch': 1.0})

And now the final step is to compute the model's performance.

In [10]:
metrics = trainer.evaluate()
metrics

  0%|          | 0/798 [00:00<?, ?it/s]

{'eval_loss': 0.011929686181247234,
 'eval_overall_precision': 0.7800192122958693,
 'eval_overall_recall': 0.7804690503652442,
 'eval_overall_f1': 0.7802440664937061,
 'eval_overall_accuracy': 0.9403374890326267,
 'eval_runtime': 56.533,
 'eval_samples_per_second': 56.463,
 'eval_steps_per_second': 14.116,
 'epoch': 1.0}

Let's try the model out with some predictions. For this we can use the `model.predict` method, which accepts either:

* A sentence as a string.
* A tokenized sentence as a list of strings.
* A list of sentences as a list of strings.
* A list of tokenized sentences as a list of lists of strings.

The method returns a list of dictionaries for each sentence, with the following keys:

* `"label"`: The string label for the found entity.
* `"score"`: The probability score indicating the model its confidence.
* `"span"`: The entity span as a string.
* `"word_start_index"` and `"word_end_index"`: Integers useful for indexing the entity from a tokenized sentence.
* `"char_start_index"` and `"char_end_index"`: Integers useful for indexing the entity from a string sentence.

In [22]:
sentences = [
    "The Ninth suffered a serious defeat at the Battle of Camulodunum under Quintus Petillius Cerialis in the rebellion of Boudica (61), when most of the foot-soldiers were killed in a disastrous attempt to relieve the besieged city of Camulodunum (Colchester).",
    "He was born in Wellingborough, Northamptonshire, where he attended Victoria Junior School, Westfield Boys School and Sir Christopher Hatton School.",
    "Nintendo continued to sell the revised Wii model and the Wii Mini alongside the Wii U during the Wii U's first release year.",
    "Dorsa has a Bachelor of Music in Composition from California State University, Northridge in 2001, Master of Music in Harpsichord Performance at Cal State Northridge in 2004, and a Doctor of Musical Arts at the University of Michigan, Ann Arbor in 2008."
]

entities_per_sentence = model.predict(sentences)

for entities in entities_per_sentence:
    for entity in entities:
        print(entity["span"], "=>", entity["label"])
    print()

Battle of Camulodunum => event
Quintus Petillius Cerialis => person
Boudica => location
Camulodunum => location
Colchester => location

Wellingborough => location
Northamptonshire => location
Victoria Junior School => organization
Westfield Boys School => organization
Sir Christopher Hatton School => organization

Nintendo => organization
Wii => product
Wii Mini => product
Wii U => product
Wii U => product

Dorsa => person
Bachelor of Music in Composition => other
California State University => organization
Northridge => location
Master of Music in Harpsichord Performance => other
Cal State Northridge => organization
Doctor of Musical Arts => other
University of Michigan => organization
Ann Arbor => location



Very impressive performance for less than 15 minutes trained! 🎉

Once trained, we can save our new model locally.

In [13]:
trainer.save_model("my_span_marker_model/checkpoint-final")

Or we can push it to the 🤗 Hub like so. I've commented it away for now to prevent people from accidentally pushing models.

In [ ]:
# trainer.push_to_hub()

If we want to use it again, we can just load it using the checkpoint or using the model name on the Hub. This is how it would be done using a local checkpoint.

In [ ]:
# model = SpanMarkerModel.from_pretrained("my_span_marker_model/checkpoint-final")

That was all! As simple as that. If we put it all together into a single script, it looks something like this:
```python
from datasets import load_dataset
from span_marker import SpanMarkerModel, Trainer
from transformers import TrainingArguments

dataset = load_dataset("DFKI-SLT/few-nerd", "supervised")
labels = dataset["train"].features["ner_tags"].feature.names

model_name = "bert-base-cased"
model = SpanMarkerModel.from_pretrained(model_name, labels=labels, model_max_length=256)

args = TrainingArguments(
    output_dir="my_span_marker_model",
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    push_to_hub=False,
    logging_steps=50,
    fp16=True,
    warmup_ratio=0.1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

trainer.train()
trainer.save_model("my_span_marker_model/checkpoint-final")
trainer.push_to_hub()

metrics = trainer.evaluate()
print(metrics)
```

With `wandb` initialized, you can enjoy their very useful training graphs straight in your browser. It ends up looking something like this.
![image](https://user-images.githubusercontent.com/37621491/228864250-dba81b82-d666-4a73-9111-ca208cc685c5.png)
![image](https://user-images.githubusercontent.com/37621491/228864399-0f0735da-ef8c-4093-9a91-51b3bd96000c.png)

Furthermore, you can use the `wandb` hyperparameter search functionality using the tutorial from the Hugging Face documentation [here](https://huggingface.co/docs/transformers/hpo_train). This transfers very well to the SpanMarker `Trainer`.